## Index
1. [Model1](#model1) - Convolution layers over word2vec 
2. [Model2](#model2) - RNN over word2vec
3. [Model3](#model3) - doc2vec using TF-IDF with word2vec
0. [Prediction](#predict) 

In [167]:
import pandas as pd       
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
import re
import lasagne
from lasagne import layers
from lasagne.layers import InputLayer,Conv1DLayer,rrelu,\
DenseLayer,ExpressionLayer,MaxPool1DLayer,GRULayer,ConcatLayer,ElemwiseMergeLayer
from theano import tensor as tt
import theano
import matplotlib.pyplot as plt
from GBM2 import GBM_KClass
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier,ExtraTreeClassifier,ExtraTreeRegressor
%matplotlib inline

In [2]:
train = pd.read_csv("/home/will/Desktop/data/Movie sentiment/labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [3]:
test = pd.read_csv("/home/will/Desktop/data/Movie sentiment/testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [124]:
unlabel = pd.read_csv("/home/will/Desktop/data/Movie sentiment/unlabeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [138]:
model = Word2Vec.load_word2vec_format('/home/will/Desktop/data/Word2Vec/GoogleNews-vectors-negative300.bin', binary=True)  

In [6]:
def review_to_wordlist(review,stop):

    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    
    words = [w for w in words if not w in stops]

    return words

In [7]:
stops = set(stopwords.words("english"))

In [8]:
X_str = []
for i in range(train.shape[0]):
    X_str.append(review_to_wordlist(train.review.iloc[i],stops))

/home/will/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/will/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [9]:
X_str_test = []
for i in range(test.shape[0]):
    X_str_test.append(review_to_wordlist(test.review.iloc[i],stops))

In [10]:
y = train.sentiment.values

In [11]:
n = y.shape[0]
y_np = np.zeros((n,2))
y_np[range(n),y]=1
y_np=y_np.astype('int64')

In [231]:
X_train = X_str[:20000]
y_train = y_np[:20000]
X_val = X_str[20000:]
y_val = y_np[20000:]

In [140]:
unlabel_str = []
for i in range(unlabel.shape[0]):
    unlabel_str.append(review_to_wordlist(unlabel.review.iloc[i],stops))

Build Batch iterator

In [12]:
def batch_iterator(X, y, WordVec, batchsize, shuffle=False, sublen=1.0):
    # inputs is a list of list, targets in np.ndarray
    # random "crop" of review, with length = int(min(sentence lenth in batch) * sublen)
    
    n = len(X)
    if shuffle:
        indices = np.random.permutation(n)
    else:
        indices = range(n)

    for start_idx in range(0, n - batchsize + 1, batchsize):
        length_ = np.zeros(batchsize,dtype='int64')
        batch_ = indices[start_idx:start_idx + batchsize]
        for i,j in enumerate(batch_): # find the min_len of batch
            length_[i] = len(X[j])
        min_len = int(length_.min() * sublen)
        sample_len = length_ - min_len + 1

        for i in range(batchsize): # sample start position
            sample_len[i] = np.random.randint(sample_len[i])
        
        X_batch = np.zeros((batchsize,300,min_len)) # 300 is the word vector's length
        
        for i in range(batchsize):
            for j in range(min_len):
                if X[batch_[i]][sample_len[i]+j] in WordVec:
                    X_batch[i,:,j] = WordVec[X[batch_[i]][sample_len[i]+j]]

        if y==None: # test 
            yield X_batch.astype('float32'), batch_
        else:
            yield X_batch.astype('float32'), y[batch_].astype('float32')

In [87]:
def batch_iterator_rnn(X, y, WordVec, batchsize, shuffle=False, sublen=1.0):
    # inputs is a list of list, targets in np.ndarray
    # random "crop" of review, with length = int(min(sentence lenth in batch) * sublen)
    
    n = len(X)
    if shuffle:
        indices = np.random.permutation(n)
    else:
        indices = range(n)

    for start_idx in range(0, n - batchsize + 1, batchsize):
        length_ = np.zeros(batchsize,dtype='int64')
        batch_ = indices[start_idx:start_idx + batchsize]
        for i,j in enumerate(batch_): # find the min_len of batch
            length_[i] = len(X[j])
        min_len = int(length_.min() * sublen)
        sample_len = length_ - min_len + 1

        for i in range(batchsize): # sample start position
            sample_len[i] = np.random.randint(sample_len[i])
        
        X_batch = np.zeros((batchsize,min_len,300)) # 300 is the word vector's length
        
        for i in range(batchsize):
            for j in range(min_len):
                if X[batch_[i]][sample_len[i]+j] in WordVec:
                    X_batch[i,j,:] = WordVec[X[batch_[i]][sample_len[i]+j]]
        
        #if np.random.rand()<flip:
        #    X_batch = X_batch[:,::-1,:]
            
        if y==None: # test 
            yield X_batch.astype('float32'), batch_
        else:
            yield X_batch.astype('float32'), y[batch_].astype('float32')

<a id='model1'></a>
1. Model1 - Conv layer over word2vec from google

In [351]:
X_tt = tt.tensor3()
y_tt = tt.matrix()
learning_rate_tt = tt.scalar()
m_tt = tt.scalar()

In [352]:
net = InputLayer(shape=(10,300,None),input_var=X_tt)
net = rrelu(Conv1DLayer(net,200,5,pad='same'))
net = rrelu(Conv1DLayer(net,200,5,pad='same'))
#net = rrelu(Conv1DLayer(net,200,3,pad='same'))
#net = rrelu(Conv1DLayer(net,200,3,pad='same'))
net = ExpressionLayer(net, lambda X: tt.max(X,2),output_shape=(10,200))
#net = rrelu(ExpressionLayer(net, lambda X: tt.mean(X,2),output_shape=(10,200)))
net = rrelu(DenseLayer(net, num_units=50))
net = DenseLayer(net, num_units=2,nonlinearity=tt.nnet.softmax)

In [353]:
Y_hat = layers.get_output(net)

In [354]:
cost = -tt.sum(tt.log(Y_hat)*y_tt)

In [355]:
err = tt.sum(tt.eq(tt.argmax(Y_hat,1),tt.argmax(y_tt,1)))

In [356]:
params = lasagne.layers.get_all_params(net, trainable=True)
updates = lasagne.updates.nesterov_momentum(
            cost, params, learning_rate=learning_rate_tt, momentum=m_tt)

In [357]:
train_fn = theano.function([X_tt, y_tt,learning_rate_tt,m_tt], updates=updates)

In [358]:
predict  = theano.function([X_tt],Y_hat)

In [359]:
loss = theano.function([X_tt, y_tt], err)

In [365]:
n_batch = 4
best_err = .0

for i in range(n_batch):
    err = 0
    
    for x_bat,y_bat in batch_iterator(X_train, y_train, model, 10, shuffle=True, sublen=1.0):
        train_fn(x_bat,y_bat,2e-5,0.9)
    
    for x_bat,y_bat in batch_iterator(X_val, y_val, model, 10, shuffle=False, sublen=1.0):
        err+=loss(x_bat,y_bat)
    
    if err > best_err:
        best_err = err
        best_para = layers.get_all_param_values(net)
    
    print 'epoch:{}, accuracy:{}'.format(i,1.0*err/y_val.shape[0])

/home/will/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


epoch:0, accuracy:0.7972
epoch:1, accuracy:0.7964
epoch:2, accuracy:0.7982
epoch:3, accuracy:0.7996


In [363]:
layers.set_all_param_values(net,best_para)

<a id='model2'></a>
Model2 - RNN over word2vec from google

In [328]:
X_tt = tt.tensor3()
y_tt = tt.matrix()
learning_rate_tt = tt.scalar()
m_tt = tt.scalar()

In [329]:
l_in = InputLayer(shape=(10,None,300),input_var=X_tt)
# first layer 
net_fw = GRULayer(l_in,150,grad_clipping=100.,\
        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
net_bw = GRULayer(l_in,150,grad_clipping=100.,backwards=True,\
        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
#net = ElemwiseMergeLayer([net_fw,net_bw],merge_function=tt.maximum)
net = rrelu(ElemwiseMergeLayer([net_fw,net_bw],merge_function=tt.add))
# second layer 
net_fw = GRULayer(net,150,grad_clipping=100.,\
        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
net_bw = GRULayer(net,150,grad_clipping=100.,backwards=True,\
        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
#net = ElemwiseMergeLayer([net_fw,net_bw],merge_function=tt.maximum)
net = rrelu(ElemwiseMergeLayer([net_fw,net_bw],merge_function=tt.add))
# third layer 
#net_fw = GRULayer(net,150,grad_clipping=100.,\
#        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
#net_bw = GRULayer(net,150,grad_clipping=100.,backwards=True,\
#        hidden_update=lasagne.layers.Gate(W_cell=None, nonlinearity=lasagne.nonlinearities.very_leaky_rectify))
#net = ElemwiseMergeLayer([net_fw,net_bw],merge_function=tt.maximum)

net = rrelu(ExpressionLayer(net, lambda X: tt.sum(X,1),output_shape=(10,150)))
#net = ExpressionLayer(net, lambda X: tt.max(X,1),output_shape=(10,150))
net = rrelu(DenseLayer(net, num_units=50))
net = DenseLayer(net, num_units=2,nonlinearity=tt.nnet.softmax)

In [330]:
Y_hat = layers.get_output(net)

In [331]:
cost = -tt.sum(tt.log(Y_hat)*y_tt)

In [332]:
err = tt.sum(tt.eq(tt.argmax(Y_hat,1),tt.argmax(y_tt,1)))

In [333]:
params = lasagne.layers.get_all_params(net, trainable=True)
updates = lasagne.updates.nesterov_momentum(
            cost, params, learning_rate=learning_rate_tt, momentum=m_tt)

In [334]:
train_fn = theano.function([X_tt, y_tt,learning_rate_tt,m_tt], updates=updates)

In [335]:
predict  = theano.function([X_tt],Y_hat)

In [336]:
loss = theano.function([X_tt, y_tt], err)

In [337]:
n_batch = 4

for i in range(n_batch):
    err = 0
    
    for x_bat,y_bat in batch_iterator_rnn(X_train, y_train, model, 10, shuffle=True, sublen=1.0):
        train_fn(x_bat,y_bat,9e-5,0.95)
    
    for x_bat,y_bat in batch_iterator_rnn(X_val, y_val, model, 10, shuffle=False, sublen=1.0):
        err+=loss(x_bat,y_bat)
        
    print 'epoch:{}, accuracy:{}'.format(i,1.0*err/y_val.shape[0])

/home/will/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:32: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


epoch:0, accuracy:0.7652
epoch:1, accuracy:0.7762
epoch:2, accuracy:0.7502
epoch:3, accuracy:0.7836


<a id='model3'></a>
Model3 - doc2vec using TF-IDF with word2vec

In [159]:
train_frq = np.zeros((25000,300),dtype='float32')
test_frq = np.zeros((25000,300),dtype='float32')

In [161]:
for i in xrange(25000):
    temp = np.zeros(300,dtype='float32')
    for j in range(len(X_str[i])):
        if X_str[i][j] in model:
            temp += model[X_str[i][j]]
    train_frq[i] = temp/len(X_str[i])

In [162]:
for i in xrange(25000):
    temp = np.zeros(300,dtype='float32')
    for j in range(len(X_str_test[i])):
        if X_str_test[i][j] in model:
            temp += model[X_str_test[i][j]]
    test_frq[i] = temp/len(X_str_test[i])

In [203]:
model1=GBM_KClass(ExtraTreeRegressor,30,1e-5,\
{'max_depth':32,'splitter':'random','max_features':0.2},25,0.2)

In [204]:
model1.fit(train_frq,y)

GBM_KClass(BaseEst=<class 'sklearn.tree.tree.ExtraTreeRegressor'>,
      BasePara={'max_features': 0.2, 'splitter': 'random', 'max_depth': 32},
      M_est=750, learnRate=1e-05, subClass=None, subFold=25)

<a id='predict'></a>
Model Submission

In [228]:
y_test = np.zeros((len(X_str_test),2))
for _ in range(10):
    for x_bat,bat in batch_iterator(X_str_test, None, model, 10, shuffle=True, sublen=1.0):
        y_test[bat] = y_test[bat] +  predict(x_bat)

In [229]:
test['sentiment'] = np.argmax(y_test,1)
test['id'] = test.id.str.replace('"','')

In [207]:
test['sentiment'] = model1.predict(test_frq)
test['id'] = test.id.str.replace('"','')

In [230]:
test.drop('review',1)\
    .to_csv(r'/home/will/Desktop/data/Movie sentiment/model1.csv',index=False)